In [1]:
import psycopg2

In [2]:
# attempting to connect with no credentials
conn = psycopg2.connect()

TypeError: missing dsn and no parameters

In [3]:
# attempting to connect to the main postres db, then create a new one from there
conn = psycopg2.connect(dbname="postgres")

In [4]:
conn.autocommit = True

In [5]:
cursor = conn.cursor()

In [6]:
CREATE_DATABASE_QUERY = "CREATE DATABASE housing_data;"

In [7]:
cursor.execute(CREATE_DATABASE_QUERY)

In [8]:
conn.close()

In [9]:
# that all seemed to work, moving on to making a schema for the sales info

# actually it seems tedious/slow to insert each row individually.  and it seems like I can avoid that if I
# am willing to use ALL of the columns, rather than making a bespoke schema for my purposes

In [10]:
conn = psycopg2.connect(dbname="housing_data")

In [11]:
cursor = conn.cursor()

In [12]:
COPY_SALES_QUERY = """COPY sales FROM STDIN WITH (FORMAT CSV)"""

In [13]:
sales_zip_file, sales_csv_file = collect_sales_data()

NameError: name 'collect_sales_data' is not defined

In [14]:
from data_collection import collect_sales_data

In [24]:
sales_zip_file, sales_csv_file = collect_sales_data()

In [16]:
cursor.copy_expert(COPY_SALES_QUERY, sales_csv_file)

UndefinedTable: relation "sales" does not exist


In [17]:
# ok, annoying, none of the docs indicate that you have to make the table before copying the CSV

In [18]:
CREATE_SALES_TABLE_QUERY = """
DROP TABLE IF EXISTS sales;
CREATE TABLE sales (
  id                 SERIAL PRIMARY KEY,
  ExciseTaxNbr       INT,
  Major              CHAR(6),
  Minor              CHAR(4),
  DocumentDate       DATE,
  SalePrice          INT,
  RecordingNbr       CHAR(14),
  Volume             CHAR(3),
  Page               CHAR(3),
  PlatNbr            CHAR(6),
  PlatType           CHAR(1),
  PlatLot            CHAR(14),
  PlatBlock          CHAR(7),
  SellerName         TEXT,
  BuyerName          TEXT,
  PropertyType       INT,
  PrincipalUse       INT,
  SaleInstrument     INT,
  AFForestLand       CHAR(1),
  AFCurrentUseLand   CHAR(1),
  AFNonProfitUse     CHAR(1),
  AFHistoricProperty CHAR(1),
  SaleReason         INT,
  PropertyClass      INT,
  SaleWarning        TEXT
);
"""

In [19]:
cursor.execute(CREATE_SALES_TABLE_QUERY)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [20]:
conn.close()

In [21]:
conn = psycopg2.connect(dbname="housing_data")

In [22]:
cursor = conn.cursor()

In [23]:
cursor.execute(CREATE_SALES_TABLE_QUERY)

In [25]:
cursor.copy_expert(COPY_SALES_QUERY, sales_csv_file)

InvalidTextRepresentation: invalid input syntax for integer: "ExciseTaxNbr"
CONTEXT:  COPY sales, line 1, column id: "ExciseTaxNbr"


In [26]:
sales_zip_file, sales_csv_file = collect_sales_data()

In [27]:
next(sales_csv_file) # Skip the header row

'"ExciseTaxNbr","Major","Minor","DocumentDate","SalePrice","RecordingNbr","Volume","Page","PlatNbr","PlatType","PlatLot","PlatBlock","SellerName","BuyerName","PropertyType","PrincipalUse","SaleInstrument","AFForestLand","AFCurrentUseLand","AFNonProfitUse","AFHistoricProperty","SaleReason","PropertyClass","SaleWarning"\n'

In [28]:
cursor.copy_expert(COPY_SALES_QUERY, sales_csv_file)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [29]:
conn.close()
conn = psycopg2.connect(dbname="housing_data")
cursor = conn.cursor()

In [30]:
cursor.copy_expert(COPY_SALES_QUERY, sales_csv_file)

UndefinedTable: relation "sales" does not exist


In [31]:
cursor.execute(CREATE_SALES_TABLE_QUERY)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [32]:
conn.close()
conn = psycopg2.connect(dbname="housing_data")
cursor = conn.cursor()

In [33]:
cursor.execute(CREATE_SALES_TABLE_QUERY)

In [34]:
cursor.copy_expert(COPY_SALES_QUERY, sales_csv_file)

StringDataRightTruncation: value too long for type character(4)
CONTEXT:  COPY sales, line 1, column minor: "03/19/1998"


In [35]:
next(sales_csv_file)

StopIteration: 

In [37]:
# huh, looks like I can't have an id column

In [38]:
CREATE_SALES_TABLE_QUERY = """
DROP TABLE IF EXISTS sales;
CREATE TABLE sales (
  ExciseTaxNbr       INT,
  Major              CHAR(6),
  Minor              CHAR(4),
  DocumentDate       DATE,
  SalePrice          INT,
  RecordingNbr       CHAR(14),
  Volume             CHAR(3),
  Page               CHAR(3),
  PlatNbr            CHAR(6),
  PlatType           CHAR(1),
  PlatLot            CHAR(14),
  PlatBlock          CHAR(7),
  SellerName         TEXT,
  BuyerName          TEXT,
  PropertyType       INT,
  PrincipalUse       INT,
  SaleInstrument     INT,
  AFForestLand       CHAR(1),
  AFCurrentUseLand   CHAR(1),
  AFNonProfitUse     CHAR(1),
  AFHistoricProperty CHAR(1),
  SaleReason         INT,
  PropertyClass      INT,
  SaleWarning        TEXT
);
"""

In [39]:
conn.close()
conn = psycopg2.connect(dbname="housing_data")
cursor = conn.cursor()

In [40]:
cursor.execute(CREATE_SALES_TABLE_QUERY)

In [41]:
sales_zip_file, sales_csv_file = collect_sales_data()
next(sales_csv_file) # Skip the header row

'"ExciseTaxNbr","Major","Minor","DocumentDate","SalePrice","RecordingNbr","Volume","Page","PlatNbr","PlatType","PlatLot","PlatBlock","SellerName","BuyerName","PropertyType","PrincipalUse","SaleInstrument","AFForestLand","AFCurrentUseLand","AFNonProfitUse","AFHistoricProperty","SaleReason","PropertyClass","SaleWarning"\n'

In [42]:
cursor.copy_expert(COPY_SALES_QUERY, sales_csv_file)

In [43]:
conn.commit()

In [44]:
# run through everything beginning to end
from data_collection import collect_sales_data
sales_zip_file, sales_csv_file = collect_sales_data()

In [45]:
from sql_utils import create_database, create_sales_table, copy_csv_to_sales_table
create_database()

ActiveSqlTransaction: DROP DATABASE cannot run inside a transaction block


In [46]:
%load_ext autoreload

In [47]:
%autoreload 2

In [48]:
create_database()

ActiveSqlTransaction: DROP DATABASE cannot run inside a transaction block


In [49]:
create_database()

ObjectInUse: database "housing_data" is being accessed by other users
DETAIL:  There is 1 other session using the database.


In [50]:
conn.close()

In [51]:
create_database()

In [52]:
# try again to run everything end-to-end

In [53]:
from data_collection import collect_sales_data
sales_zip_file, sales_csv_file = collect_sales_data()

In [54]:
from sql_utils import create_database, create_sales_table, copy_csv_to_sales_table
create_database()

In [55]:
conn = psycopg2.connect(dbname="housing_data")

In [56]:
create_sales_table(conn)

In [57]:
copy_csv_to_sales_table(conn, sales_csv_file)

In [58]:
sales_zip_file.close()
sales_csv_file.close()
conn.close()

In [59]:
# okay, that all ran, let's see if we can read stuff!
conn = psycopg2.connect(dbname="housing_data")

In [60]:
import pandas as pd

In [61]:
pd.read_sql_query("SELECT * FROM sales LIMIT 5;", conn)

,excisetaxnbr,major,minor,documentdate,saleprice,recordingnbr,volume,page,platnbr,plattype,...,propertytype,principaluse,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning
0,1600768,330405,0100,1998-03-19,215000,199803251689,145,039,330405,C,...,2,2,3,N,N,N,N,1,3,
1,2413752,868146,0030,2009-09-11,0,20091022001461,,,,,...,3,2,15,N,N,N,N,11,3,18 31 38
2,1939480,258190,0265,2003-02-06,0,20030214003390,,,,,...,3,6,15,N,N,N,N,10,8,31 51
3,2999169,919715,0200,2019-07-08,192000,20190712001080,,,,,...,3,2,3,N,N,N,N,1,3,
4,2220242,334330,1343,2006-05-30,0,20060706002163,,,,,...,1,6,3,N,N,N,N,18,7,11 31


In [62]:
pd.read_sql_query("SELECT * FROM sales WHERE date_part('year', DocumentDate) = 2018 LIMIT 5;", conn)

,excisetaxnbr,major,minor,documentdate,saleprice,recordingnbr,volume,page,platnbr,plattype,...,propertytype,principaluse,saleinstrument,afforestland,afcurrentuseland,afnonprofituse,afhistoricproperty,salereason,propertyclass,salewarning
0,2947815,683782,0300,2018-07-23,280001,20180816000825,,,,,...,0,6,21,N,N,N,N,1,8,14 31
1,2937935,873178,0100,2018-06-20,185000,20180625000560,,,,,...,14,2,3,N,N,N,N,1,3,
2,2912844,387610,0640,2018-01-24,0,,,,,,...,3,6,15,N,N,N,N,5,8,12 18 31
3,2952652,281809,0020,2018-09-13,165000,20180914001119,,,,,...,3,2,3,N,N,N,N,1,3,15
4,2936365,260779,0010,2018-06-07,650000,20180615001343,,,,,...,3,2,3,N,N,N,N,1,3,


In [63]:
# ok, this is working, let's make another commit

In [64]:
# time for buildings and parcels!

In [65]:
CREATE_BUILDINGS_TABLE_QUERY = """
    DROP TABLE IF EXISTS buildings;
    CREATE TABLE BUILDINGS (
        Major              CHAR(6),
        Minor              CHAR(4),
        BldgNbr            INT,
        NbrLivingUnits     INT,
        Address            TEXT,
        BuildingNumber     CHAR(5),
        Fraction           CHAR(3),
        DirectionPrefix    CHAR(2),
        StreetName         CHAR(25),
        StreetType         CHAR(6),
        DirectionSuffix    CHAR(2),
        ZipCode            CHAR(10),
        Stories            INT,
        BldgGrade          INT,
        BldgGradeVar       INT,
        SqFt1stFloor       INT,
        SqFtHalfFloor      INT,
        SqFt2ndFloor       INT,
        SqFtUpperFloor     INT,
        SqFtUnfinFull      INT,
        SqFtUnfinHalf      INT,
        SqFtTotLiving      INT,
        SqFtTotBasement    INT,
        SqFtFinBasement    INT,
        FinBasementGrade   INT,
        SqFtGarageBasement INT,
        SqFtGarageAttached INT,
        DaylightBasement   CHAR(1),
        SqFtOpenPorch      INT,
        SqFtEnclosedPorch  INT,
        SqFtDeck           INT,
        HeatSystem         INT,
        HeatSource         INT,
        BrickStone         INT,
        ViewUtilization    CHAR(1),
        Bedrooms           INT,
        BathHalfCount      INT,
        Bath3qtrCount      INT,
        BathFullCount      INT,
        FpSingleStory      INT,
        FpMultiStory       INT,
        FpFreestanding     INT,
        FpAdditional       INT,
        YrBuilt            INT,
        YrRenovated        INT,
        PcntComplete       INT,
        Obsolescence       INT,
        PcntNetCondition   INT,
        Condition          INT,
        AddnlCost          INT
    );
"""

In [66]:
cursor = conn.cursor()

In [67]:
cursor.execute(CREATE_BUILDINGS_TABLE_QUERY)

In [68]:
from data_collection import collect_building_data
buildings_zip_file, buildings_csv_file = collect_building_data()

In [69]:
next(buildings_csv_file)

'"Major","Minor","BldgNbr","NbrLivingUnits","Address","BuildingNumber","Fraction","DirectionPrefix","StreetName","StreetType","DirectionSuffix","ZipCode","Stories","BldgGrade","BldgGradeVar","SqFt1stFloor","SqFtHalfFloor","SqFt2ndFloor","SqFtUpperFloor","SqFtUnfinFull","SqFtUnfinHalf","SqFtTotLiving","SqFtTotBasement","SqFtFinBasement","FinBasementGrade","SqFtGarageBasement","SqFtGarageAttached","DaylightBasement","SqFtOpenPorch","SqFtEnclosedPorch","SqFtDeck","HeatSystem","HeatSource","BrickStone","ViewUtilization","Bedrooms","BathHalfCount","Bath3qtrCount","BathFullCount","FpSingleStory","FpMultiStory","FpFreestanding","FpAdditional","YrBuilt","YrRenovated","PcntComplete","Obsolescence","PcntNetCondition","Condition","AddnlCost"\n'

In [70]:
COPY_BUILDINGS_QUERY = """COPY buildings FROM STDIN WITH (FORMAT CSV)"""

In [71]:
cursor.copy_expert(COPY_BUILDINGS_QUERY, buildings_csv_file)

InvalidTextRepresentation: invalid input syntax for integer: "1.5"
CONTEXT:  COPY buildings, line 2, column stories: "1.5"


In [72]:
# ok, looks like we are going to have to deal with different types of "numbers"

In [73]:
CREATE_BUILDINGS_TABLE_QUERY = """
    DROP TABLE IF EXISTS buildings;
    CREATE TABLE BUILDINGS (
        Major              CHAR(6),
        Minor              CHAR(4),
        BldgNbr            INT,
        NbrLivingUnits     INT,
        Address            TEXT,
        BuildingNumber     CHAR(5),
        Fraction           CHAR(3),
        DirectionPrefix    CHAR(2),
        StreetName         CHAR(25),
        StreetType         CHAR(6),
        DirectionSuffix    CHAR(2),
        ZipCode            CHAR(10),
        Stories            REAL,
        BldgGrade          INT,
        BldgGradeVar       INT,
        SqFt1stFloor       INT,
        SqFtHalfFloor      INT,
        SqFt2ndFloor       INT,
        SqFtUpperFloor     INT,
        SqFtUnfinFull      INT,
        SqFtUnfinHalf      INT,
        SqFtTotLiving      INT,
        SqFtTotBasement    INT,
        SqFtFinBasement    INT,
        FinBasementGrade   INT,
        SqFtGarageBasement INT,
        SqFtGarageAttached INT,
        DaylightBasement   CHAR(1),
        SqFtOpenPorch      INT,
        SqFtEnclosedPorch  INT,
        SqFtDeck           INT,
        HeatSystem         INT,
        HeatSource         INT,
        BrickStone         INT,
        ViewUtilization    CHAR(1),
        Bedrooms           INT,
        BathHalfCount      INT,
        Bath3qtrCount      INT,
        BathFullCount      INT,
        FpSingleStory      INT,
        FpMultiStory       INT,
        FpFreestanding     INT,
        FpAdditional       INT,
        YrBuilt            INT,
        YrRenovated        INT,
        PcntComplete       INT,
        Obsolescence       INT,
        PcntNetCondition   INT,
        Condition          INT,
        AddnlCost          INT
    );
"""

In [74]:
conn.close()

In [75]:
conn = psycopg2.connect(dbname="housing_data")
cursor = conn.cursor()
cursor.execute(CREATE_BUILDINGS_TABLE_QUERY)

In [76]:
buildings_zip_file, buildings_csv_file = collect_building_data()
next(buildings_csv_file)

'"Major","Minor","BldgNbr","NbrLivingUnits","Address","BuildingNumber","Fraction","DirectionPrefix","StreetName","StreetType","DirectionSuffix","ZipCode","Stories","BldgGrade","BldgGradeVar","SqFt1stFloor","SqFtHalfFloor","SqFt2ndFloor","SqFtUpperFloor","SqFtUnfinFull","SqFtUnfinHalf","SqFtTotLiving","SqFtTotBasement","SqFtFinBasement","FinBasementGrade","SqFtGarageBasement","SqFtGarageAttached","DaylightBasement","SqFtOpenPorch","SqFtEnclosedPorch","SqFtDeck","HeatSystem","HeatSource","BrickStone","ViewUtilization","Bedrooms","BathHalfCount","Bath3qtrCount","BathFullCount","FpSingleStory","FpMultiStory","FpFreestanding","FpAdditional","YrBuilt","YrRenovated","PcntComplete","Obsolescence","PcntNetCondition","Condition","AddnlCost"\n'

In [77]:
cursor.copy_expert(COPY_BUILDINGS_QUERY, buildings_csv_file)

In [78]:
conn.commit()

In [79]:
pd.read_sql_query("SELECT * FROM buildings LIMIT 5;", conn)

,major,minor,bldgnbr,nbrlivingunits,address,buildingnumber,fraction,directionprefix,streetname,streettype,...,fpmultistory,fpfreestanding,fpadditional,yrbuilt,yrrenovated,pcntcomplete,obsolescence,pcntnetcondition,condition,addnlcost
0,012000,0032,1,1,830 8TH AVE S 98033,830,,,8TH,AVE,...,0,0,0,1981,0,0,0,0,3,0
1,012000,0260,1,1,341 8TH ST S 98033,341,,,8TH,ST,...,0,0,0,1928,2007,0,0,0,3,0
2,012000,0262,1,1,712 5TH AVE S 98033,712,,,5TH,AVE,...,0,0,0,2010,0,0,0,0,3,0
3,012005,9035,1,1,39525 AUBURN-ENUMCLAW RD SE 98092,39525,,,AUBURN-ENUMCLAW,RD,...,0,0,0,1917,0,0,0,0,1,0
4,012006,9079,1,1,39007 272ND AVE SE 98022,39007,,,272ND,AVE,...,0,0,0,1990,0,0,0,0,4,0


In [80]:
conn.close()

In [81]:
# run through the whole buildings flow
from data_collection import collect_building_data
buildings_zip_file, buildings_csv_file = collect_building_data()

In [82]:
from sql_utils import create_database, create_buildings_table, copy_csv_to_buildings_table
create_database()

In [83]:
conn = psycopg2.connect(dbname="housing_data")

In [84]:
create_buildings_table(conn)

In [85]:
copy_csv_to_buildings_table(conn, buildings_csv_file)

In [86]:
buildings_zip_file.close()
buildings_csv_file.close()
conn.close()

In [87]:
# test whether it worked
conn = psycopg2.connect(dbname="housing_data")
pd.read_sql_query("SELECT * FROM buildings LIMIT 5;", conn)

,major,minor,bldgnbr,nbrlivingunits,address,buildingnumber,fraction,directionprefix,streetname,streettype,...,fpmultistory,fpfreestanding,fpadditional,yrbuilt,yrrenovated,pcntcomplete,obsolescence,pcntnetcondition,condition,addnlcost
0,012000,0032,1,1,830 8TH AVE S 98033,830,,,8TH,AVE,...,0,0,0,1981,0,0,0,0,3,0
1,012000,0260,1,1,341 8TH ST S 98033,341,,,8TH,ST,...,0,0,0,1928,2007,0,0,0,3,0
2,012000,0262,1,1,712 5TH AVE S 98033,712,,,5TH,AVE,...,0,0,0,2010,0,0,0,0,3,0
3,012005,9035,1,1,39525 AUBURN-ENUMCLAW RD SE 98092,39525,,,AUBURN-ENUMCLAW,RD,...,0,0,0,1917,0,0,0,0,1,0
4,012006,9079,1,1,39007 272ND AVE SE 98022,39007,,,272ND,AVE,...,0,0,0,1990,0,0,0,0,4,0
